Cómo comarar una descomposición con otra luego de un outliear. Asumiendo que se tiene una descomposición "buena"; DEC2020, podemos ajustar con los dos modelos DEC2024 y DEC2024, y comparar MSE entre ellas, y entre la tasa oficial (no ajustada). Se puede hacer esto para comparar todos los modelos pre y post pandemia, y podemos obtener una base de erores comparando modelos dentro de los mismos periodos.

In [68]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

Será necesario convertir los indices desde tuplas (Año, Trimeste Móvil) a Datetimeindex para indexar.

In [69]:
from sklearn.metrics import mean_squared_error

def compared_series(pre_serie:pd.Series, pos_serie:pd.Series, outlier_start, outlier_end):
    """

    Parametros:
    pre_serie: Serie temporal antes del outliear
    pos_serie: Serie temporal después del outliear

    Las series deben tener el mismo multi indice ej: (Año, Trimestre) y deben estar orderanas de forma descendente. 
    """
    mse = mean_squared_error(pre_serie, pos_serie)
    mse_pre = mean_squared_error(pre_serie.loc[:outlier_start], pos_serie.loc[:outlier_start])
    print("MSE Series {pre}: ", mse)
    print("MSE pre outlier:", mse_pre)
    return mse, mse_pre

def mse_seasonality(pre_serie:pd.Series, pos_serie:pd.Series, outlier_start, outlier_end, pre_serie_comp:pd.Series=None, pos_serie_comp:pd.Series=None):
    """
    Parametros:
    pre_serie: Serie temporal antes del outliear
    pos_serie: Serie temporal después del outliear
    pre_serie_comp: Serie temporal comparativa antes del outliear
    pos_serie_comp: Serie temporal comparativa después del outliear

    Las series deben tener el mismo multi indice ej: (2020, "Abr- Jun"), una sola columna y deben estar orderanas de forma descendente. 

    Returns:
    model_diff: MSE de las series pre y pos pandemia menos el MSE encontrado en dos series pre pandemia
    pre_mse_comp:
    pos_mse_comp:
    """
    if pre_serie_comp is None:
        pre_serie_comp = pre_serie
    if pos_serie_comp is None:
        pos_serie_comp = pos_serie

    pos_serie = pos_serie.fillna(pos_serie_comp) # Se reemplaza solo con la serie similar
    pos_serie_comp = pos_serie_comp.fillna(pos_serie) # Lo mismo para anular la contribución con lo que resta de la serie similar    
    pos_mse_comp = mean_squared_error(pos_serie.loc[outlier_end:], pos_serie_comp.loc[outlier_end:])

    pre_serie = pre_serie.fillna(pre_serie_comp).fillna(pos_serie) # Se reemplaza para con la serie similar luego con la serie a comparar para anular la contribución
    pre_serie_comp = pre_serie_comp.fillna(pre_serie).fillna(pos_serie) # Lo mismo para anular la contribución con lo que resta de la serie similar
    pre_mse_comp = mean_squared_error(pre_serie.loc[:outlier_start], pre_serie_comp.loc[:outlier_start])
    pre_mse = mean_squared_error(pre_serie.loc[:outlier_start], pos_serie.loc[:outlier_start])
    return pre_mse/pre_mse_comp, pre_mse_comp, pos_mse_comp

In [70]:
(np.arange(12,0,-1) - 7)%12 +1 

array([ 6,  5,  4,  3,  2,  1, 12, 11, 10,  9,  8,  7])

In [71]:
def date_to_index(data):
    df = data.copy()
    trim_date = dict(zip(df[("Trimestre",np.nan)].unique(), (np.arange(12,0,-1) - 7)%12+1)) # Ej: {'Dic - Feb': 1001, 'Nov - Feb': 1012}.  
    trim_date = {k:f'{v:02}' for k,v in trim_date.items()} # Anteriormente se sumo 1000 para anteponer un 0 al número, luego de convertido a string.
    df[("Trimestre",np.nan)] = df[("Trimestre",np.nan)].apply(lambda trim: trim_date[trim])
    df[("Trimestre", np.nan)]
    df["Date"] = df[["Año", "Trimestre"]].agg(lambda row: "-".join(row.astype("string")) + "-01", axis=1)
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    df.index = df["Date"]
    df.drop(columns=[("Año",np.nan), ("Trimestre",np.nan), ("Date","")], inplace=True)
    df.sort_index(inplace=True)
    
    return df

data = pd.read_excel(io="../data/ine/ajuste_estacional_historico.xlsx", sheet_name="tasa_as")
data = data.loc[3:172]
data.columns= pd.MultiIndex.from_arrays(data.iloc[0:2].to_numpy())
data = data.loc[6:].reset_index(drop=True)
data
0

0

In [72]:
data = date_to_index(data=data)
data

,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,...,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada,Tasa oficial,Tasa ajustada
,NDE2019,NDE2019,DEF2020,DEF2020,EFM2020,EFM2020,FMA2020,FMA2020,MAM2020,MAM2020,...,EFM2024,EFM2024,FMA2024,FMA2024,MAM2024,MAM2024,AMJ2024,AMJ2024,MJJ2024,MJJ2024
Date,,,,,,,,,,,,,,,,,,,,,
2010-08-01,8.1,7.9,8.124569,7.926932,8.124569,7.925845,8.124569,7.925795,8.124569,7.926,...,8.124569,7.928278,8.124569,7.928196,8.124569,7.928154,8.124569,7.928121,8.124569,7.928104
2010-09-01,7.8,7.7,7.814697,7.704444,7.814697,7.704655,7.814697,7.706665,7.814697,7.706946,...,7.814697,7.702712,7.814697,7.701814,7.814697,7.702325,7.814697,7.701711,7.814697,7.696071
2010-10-01,7.2,7.4,7.22505,7.427053,7.22505,7.428384,7.22505,7.426959,7.22505,7.42707,...,7.22505,7.418691,7.22505,7.419761,7.22505,7.418367,7.22505,7.418054,7.22505,7.41353
2010-11-01,7.2,7.5,7.210129,7.531558,7.210129,7.531692,7.210129,7.533507,7.210129,7.533247,...,7.210129,7.528851,7.210129,7.528725,7.210129,7.52886,7.210129,7.528905,7.210129,7.529803
2010-12-01,7.4,7.6,7.441762,7.646923,7.441762,7.647194,7.441762,7.6478,7.441762,7.647753,...,7.441762,7.666947,7.441762,7.666545,7.441762,7.667077,7.441762,7.667357,7.441762,7.67283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.677778,8.455696,8.677778,8.448077,8.677778,8.437456,8.677778,8.443967,8.677778,8.446938
2024-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.523784,8.359039,8.523784,8.34056,8.523784,8.357063,8.523784,8.366389


In [73]:
save = data.loc[:,("Tasa ajustada", "MJJ2024")].reset_index()
save.columns = ['ds', 'to']
save.to_csv("../data/endogena/to202406.csv", index=False)

In [74]:
diff, premse, posmse = mse_seasonality(
    pre_serie=data[("Tasa ajustada", "NDE2019")],
    pos_serie=data[("Tasa ajustada", "MJJ2024")],
    pre_serie_comp=data[("Tasa ajustada", "EFM2020")],
    pos_serie_comp=data[("Tasa ajustada", "MJJ2023")],
    outlier_start="2020-03-01", outlier_end="2022-03-01"
    )

C:\Users\aurib\AppData\Local\Temp\ipykernel_25744\16537595.py:38: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\aurib\AppData\Local\Temp\ipykernel_25744\16537595.py:39: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\aurib\AppData\Local\Temp\ipykernel_25744\16537595.py:42: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\U

In [75]:
print("diferencia: ", diff)
print("MSE previo al outlier", premse)
print("MSE posterior al outlier", posmse)

diferencia:  1.1730499121604434
MSE previo al outlier 0.001112485417431904
MSE posterior al outlier 0.0026937396441335027


In [76]:
pdata = data.copy()
pdata.columns = pd.Index(map(str.strip, pdata.columns.get_level_values(0) + " " + pdata.columns.get_level_values(1)))
fig = px.line(pdata, y=["Tasa oficial DEF2020",  "Tasa ajustada DEF2020", "Tasa oficial DEF2024", "Tasa ajustada DEF2024"], 
              title="Tasa de desocupación y sus ajustes estacionales según DEF2020 y DEF2024",
              markers=True)
fig

In [77]:
from statsmodels.tsa.x13 import x13_arima_analysis as x13
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from os import path

x13as_path = path.abspath("C:/Program Files/x13as")
x13o = x13(
    endog=data.loc[:,("Tasa oficial", "DEF2024")].dropna(),
    maxorder=(1,1),
    forecast_periods=24,
    x12path=x13as_path,
    outlier=False)

In [78]:
print(x13o.seasadj)

Date
2010-08-01    7.973291
2010-09-01    7.639164
2010-10-01    7.428068
2010-11-01    7.524985
2010-12-01    7.645171
                ...   
2023-09-01    8.873446
2023-10-01    9.001044
2023-11-01    8.983292
2023-12-01    8.833883
2024-01-01    8.724162
Name: seasadj, Length: 162, dtype: float64


In [79]:
mean_squared_error(x13o.seasadj, data.loc[:,("Tasa ajustada", "DEF2024")].dropna())

np.float64(0.008792990951350228)

In [80]:
pdata = data.copy()
pdata.columns = pd.Index(map(str.strip, pdata.columns.get_level_values(0) + " " + pdata.columns.get_level_values(1)))
pdata = pd.concat([pdata.loc[:,["Tasa ajustada DEF2024", "Tasa oficial DEF2024"]].dropna(), x13o.seasadj], axis=1)\
    .convert_dtypes('float64')\
        .rename({"seasadj":"Tasa ajustada DEF2024 SM", "Tasa ajustada DEF2024":"Tasa ajustada DEF2024 INE", "Tasa oficial DEF2024":"Tasa oficial DEF2024 INE"}, axis=1)
fig = px.line(pdata, y=["Tasa ajustada DEF2024 INE", "Tasa ajustada DEF2024 SM", "Tasa oficial DEF2024 INE"], 
              title="Tasa de desocupación y sus ajustes estacionales según DEF2020 y DEF2024",
              markers=True)
fig

In [81]:
generic_layouts = {"legend_title":"Serie temporal:", "yaxis_title":"<b>Tasa</b>","xaxis_title":"<b>Fecha</b>",
                    "font":dict(
                        family="Courier New, monospace",
                        size=16,
                        color="Black",
                        variant="small-caps"
                        ),
                    "legend":dict(
                        # orientation="h",
                        yanchor="bottom",
                        y=1.0,
                        xanchor="right",
                        x=1.0,
                        bgcolor="LightBlue"
                        )
                    }

In [82]:
pdata = data.copy()
pdata.columns = pd.Index(map(str.strip, pdata.columns.get_level_values(0) + " " + pdata.columns.get_level_values(1)))
pdata = pdata.loc[:,"Tasa oficial DEF2024"].dropna()#.rename({"Tasa oficial DEF2024":"Tasa oficial DEF2024"}, axis=1)
# print(pdata.columns)
fig = px.line(x=pdata.index, y=pdata, 
              title="Tasa de desocupación, trimestre DEF2024",
              markers=True,
              )
fig.update_layout(**generic_layouts)
fig

In [83]:
pan = pd.read_csv("D:\FCFM\Primavera2024\ProyectoINE\proyectomds_ine\data\pandemia\defunciones_covid19_2020_2024.csv", sep=";")
pan["FECHA_DEF"] = pd.to_datetime(pan["FECHA_DEF"], format="%Y-%m-%d")
pan.sort_values(by="FECHA_DEF", inplace=True)
pan.index = pan["FECHA_DEF"]

In [84]:
countpan = pan.groupby(by=[pan.index.year, pan.index.month]).count()
countpan.index = pd.to_datetime(countpan.index.to_series().apply(lambda tuple: "-".join([str(t) for t in tuple])+"-01"))


In [85]:
generic_layouts = {"legend_title":"Serie temporal:", "yaxis_title":"<b>Cantidad</b>","xaxis_title":"<b>Fecha</b>",
                    "font":dict(
                        family="Courier New, monospace",
                        size=16,
                        color="Black",
                        variant="small-caps"
                        ),
                    "legend":dict(
                        # orientation="h",
                        yanchor="bottom",
                        y=1.0,
                        xanchor="right",
                        x=1.0,
                        bgcolor="LightBlue"
                        )
                    }

fig = px.line(countpan, y="EDAD_CANT", 
              title="<b>Defunciones Mensuales COVID-19</b>",
              markers=True)
fig.update_layout(**generic_layouts
)
fig

In [86]:
generic_layouts = {"legend_title":"Serie temporal:", "yaxis_title":"<b>Tasa de Desocupación</b>","xaxis_title":"<b>Fecha</b>","yaxis_range":[5.5,13.5],
                    "font":dict(
                        family="Courier New, monospace",
                        size=16,
                        color="Black",
                        variant="small-caps"
                        ),
                    "legend":dict(
                        # orientation="h",
                        yanchor="bottom",
                        y=1.0,
                        xanchor="right",
                        x=1.0,
                        bgcolor="LightBlue"
                        )
                    }

def composed_adjustment(data:pd.DataFrame, start="NDE2019", end="2022-06-01",outlier_span=29, model="AMJ2023", graphs=[True,True,True,True,True]):
    tri_map = {"DEF":1, "EFM":2, "FMA":3, "MAM":4, "AMJ":5, "MJJ":6, "JJA":7, "JAS":8, "ASO":9, "SON":10, "OND":11, "NDE":12}
    pre_pan = data.loc[:,("Tasa oficial", start)].copy().astype('float64').dropna() # Modelo antes de la pandemia según OMS
    pre_pan.rename(" ".join(pre_pan.name), inplace=True)
    pos_pan = data.loc[:,("Tasa oficial", model)].copy().astype('float64').dropna()
    pos_pan.rename(" ".join(pos_pan.name), inplace=True)
    justpos_pan = pos_pan.loc["2020-01-01":].copy() # Modelo terminada la alerta de pandemia según OMS

    # Modelo SARIMAX para predecir periodo pandemia
    sari = SARIMAX(
        endog=pre_pan,
        order=(4,1,4),
        seasonal_order=(2,1,2,12),
        freq='MS'
        )
    results = sari.fit()
    fore29 = results.forecast(steps=outlier_span) # Desde DEF2020 hasta AMJ2022 fecha donde plan paso a paso
    fore29.rename("Tasa pronosticada periodo Pandemia", inplace=True)
    fore = pd.concat([pre_pan, fore29, justpos_pan], axis=1)
    
    if graphs[0]:
        fig = px.line(fore, y=[f"Tasa oficial {start}",  "Tasa pronosticada periodo Pandemia"], 
                    title="<b>Tasa oficial y su pronostico para pediodo Pandemia</b>",
                markers=True)
        fig.update_layout(**generic_layouts)
        fig.show()
    if graphs[1]:
        fig = px.line(fore, y=[f"Tasa oficial {start}",  "Tasa pronosticada periodo Pandemia", f"Tasa oficial {model}"], 
                    title="<b>Tasa oficial y su pronostico pediodo Pandemia</b>",
                    markers=True)
        fig.update_layout(**generic_layouts)
        fig.show()
    compose = fore[f"Tasa oficial {start}"].fillna(fore["Tasa pronosticada periodo Pandemia"]).fillna(fore[f"Tasa oficial {model}"]).rename(f"Serie compuesta (DEF2020, AMJ2022)")
    fore_new = pd.concat([compose, justpos_pan], axis=1)
    if graphs[2]:
        fig = px.line(fore_new, y=[f"Tasa oficial {model}", f"Serie compuesta (DEF2020, AMJ2022)"],
                    title="<b>Tasa compuesta, tasa oficial-pronostico-oficial (prepandemia-pronostico-pospandemia) para periodo Pandemia</b>",
                    markers=True)
        fig.update_layout(**generic_layouts)
        fig.show()

    # Modelo X13-SARIMA
    # Desestacionalización Serie compuesta por parte OFICIAL-PRONOSTICO-OFICIAL (Prepandemia-Pandemia-Pospandemia)
    x13comp = x13(
        endog=fore_new[f"Serie compuesta (DEF2020, AMJ2022)"].dropna(),
        maxorder=(1,1),
        x12path=x13as_path,
        outlier=False)
    # Predicción Serie Oficial
    x13real = x13(
        endog=pos_pan.dropna(),
        maxorder=(1,1),
        x12path=x13as_path,
        outlier=False)
    comp_adj = x13comp.seasadj.rename(f"Ajuste estacional compuesto {model}")
    real_adj = x13real.seasadj.rename(f"Ajuste estacional real {model}")

    if graphs[3]:    
        fig = px.line(pd.concat([comp_adj, real_adj], axis=1),
                    title=f"<b>Tasa de desocupación y sus ajustes estacionales según serie compuesta y {model}</b>",
                    markers=True)
        fig.update_layout(**generic_layouts)
        fig.show()
    
    if graphs[4]:
        diff = (comp_adj-real_adj).rename("Residuo")
        fig = px.line(diff,
                    title=f"<b>Diferencia en desetacionalización real y compuesta para {model}</b>",
                    markers=True)
        fig.update_layout(**generic_layouts)
        fig.update_layout(yaxis_range=[-5, 0.5])
        fig.show()
        
    model_diff = mean_squared_error(comp_adj.loc[:end], real_adj.loc[:end]) # MSE sin contabilizar directamente nuevos datos
    print("MSE para desestacionalización Tasa compuesta y Tasa real: ", model_diff)
    return model_diff


In [87]:
# modelos = data["Tasa oficial"].columns[-25:].to_numpy()
# model_diffs = {}
# for modelo in modelos:
#     model_diffs[modelo] = composed_adjustment(data, model=modelo, graphs=[False,False,False,False,True])


In [88]:
# generic_layouts = {"legend_title":"Serie temporal:", "yaxis_title":"<b>Residuo</b>","xaxis_title":"<b>Modelo</b>",
#                     "font":dict(
#                         family="Courier New, monospace",
#                         size=16,
#                         color="Black",
#                         variant="small-caps"
#                         ),
#                     "legend":dict(
#                         # orientation="h",
#                         yanchor="bottom",
#                         y=1.0,
#                         xanchor="right",
#                         x=1.0,
#                         bgcolor="LightBlue"
#                         )
#                     }

# diffs = pd.DataFrame.from_dict(model_diffs, orient="index")
# diffs.columns = ["<b>Residuos de destacianalización por modelo</b>"]
# fig = px.line(diffs,
#             title=f"<b>Diferencia en desetacionalización real y compuesta para cada modelo</b>",
#             markers=True)
# fig.update_layout(**generic_layouts)
# fig.show()

In [89]:
from statsmodels.tsa.x13 import x13_arima_analysis
from statsmodels.tsa.statespace.sarimax import SARIMAX
from plotly import graph_objects as go

import warnings
from statsmodels.tools.sm_exceptions import X13Warning, ConvergenceWarning, ValueWarning, ModelWarning
warnings.simplefilter('ignore', category=X13Warning)
warnings.simplefilter('ignore', category=ConvergenceWarning)
warnings.simplefilter('ignore', category=UserWarning)
warnings.simplefilter('ignore', category=ValueWarning)



class OutlierAnalisys():
    x13as_path = path.abspath("C:/Program Files/x13as")
    
    def __init__(self, time_serie:pd.Series, start_date:pd.Timestamp=None, end_date:pd.Timestamp=None, forecast_model=SARIMAX, exogenous=[]) -> None:
        self.serie = time_serie
        self.comp_serie = None
        self.comp_adj = None
        self.real_adj = None
        self.start = start_date
        self.end = end_date
        self.forecast_model = forecast_model
        self.outlier_period = (self.end.year-self.start.year)*12 + (self.end.month - self.start.month)

    def forecast(self, periods, serie=None, model=None):
        serie = self.serie if serie is None else serie
        model = self.forecast_model if model is None else model

        if model.__name__=='SARIMAX':
            sari = model(
                endog=serie,
                order=(4,1,4),
                seasonal_order=(2,1,2,12),
                freq='MS'
                )
            results = sari.fit()
            if periods > 0:
                fore = results.forecast(steps=periods)

        return fore

    def compose_serie(self, model, serie=None):
        serie = self.serie if serie is None else serie
        model = self.forecast_model if model is None else model

        pre_out = serie.loc[:self.start].copy()
        post_out = serie.loc[self.end:].copy()
        
        # Modelo SARIMAX para predecir periodo pandemia
        fore = self.forecast(self.outlier_period, pre_out, model)
        comp_serie = pre_out.reindex(serie.index)
        comp_serie = comp_serie.fillna(fore).fillna(post_out)

        self.comp_serie = comp_serie if serie is None else self.comp_serie
        return comp_serie
    
    def seasonality_diff(self, seasonal_model=x13_arima_analysis, serie=None,  forecast_model=None, mse_limit=None):
        serie = self.serie if serie is None else serie
        forecast_model = self.forecast_model if forecast_model is None else forecast_model

        comp_serie = self.compose_serie(forecast_model, serie)
        
        # Modelo X13-SARIMA
        # Desestacionalización Serie compuesta por parte OFICIAL-PRONOSTICO-OFICIAL (Prepandemia-Pandemia-Pospandemia)
        if seasonal_model.__name__== 'x13_arima_analysis':
            x13comp = seasonal_model(
                endog=comp_serie,
                maxorder=(1,1),
                x12path=x13as_path,
                outlier=False)
            # Predicción Serie Oficial
            x13real = seasonal_model(
                endog=serie,
                maxorder=(1,1),
                x12path=x13as_path,
                outlier=False)
        
            comp_adj = x13comp.seasadj
            real_adj = x13real.seasadj
            self.comp_adj = comp_adj if serie is None else self.comp_adj
            self.real_adj = real_adj if serie is None else self.real_adj

        mse_limit = slice(mse_limit)
        model_diff = mean_squared_error(comp_adj.loc[mse_limit], real_adj.loc[mse_limit]) # MSE sin contabilizar directamente nuevos datos
        print("MSE para desestacionalización Tasa compuesta y Tasa real: ", model_diff)
        
        return model_diff
    

    def plot(self, mode):
        plots = {
            'composed':go.Scatter(x=self.comp_serie.index, y=self.comp_serie, name='Composed'), 
            'real':go.Scatter(x=self.serie.index, y=self.serie, name='Real'), 
            'adjusted composed':go.Scatter(x=self.comp_adj.index, y=self.comp_adj, name='Adjusted Composed'),
            'adjusted real':go.Scatter(x=self.real_adj.index, y=self.real_adj, name='Adjusted Real'),
            'adjusted diff':go.Scatter(x=self.real_adj.index, y=self.real_adj-self.comp_adj, name='Difference on Adjustment'),
            'diff':go.Scatter(x=self.serie.index, y=self.serie-self.comp_serie, name='Real Difference')
            }
        fig = go.Figure()
        if mode=='all':
            for n, p in plots.items():
                fig.add_traces(p)
        elif mode=='adjusted':
            for n, p in plots.items():
                if 'adjusted' in n and 'diff' not in n:
                    fig.add_traces(p)
        elif mode=='normal':
            for n, p in plots.items():
                if 'adjusted' not in n and 'diff' not in n:
                    fig.add_traces(p)
        elif mode=='diff':
            for n, p in plots.items():
                if 'adjusted' not in n and 'diff' in n:
                    fig.add_traces(p)
        else:
            fig.add_traces(plots[mode])
    
    def model_evolution(self, seasonal_model=x13_arima_analysis):
        self.mses = []
        last_date = self.end
        while last_date in self.serie.index:
            cropped = self.serie[:last_date + pd.DateOffset(months=1)]
            self.mses.append(self.seasonality_diff(serie=cropped, seasonal_model=seasonal_model, mse_limit=self.start))
            last_date = last_date + pd.DateOffset(months=1)
    
    def plot_evol(self):
        fig = go.Figure(go.Scatter(x=self.serie[self.end:].index, y=self.mses))
        fig.show()
        
    
        

In [90]:
s = pd.Series(np.sin(np.linspace(0, 50, 160) + 2)+ 3*np.cos(np.linspace(0, 100, 160) + 0.6*np.random.randn(160)))
s.index = pd.date_range(start="2010-09-01", freq="MS", periods=len(s))
fig = go.Figure(go.Scatter(x=s.index, y=s))
fig.show()
# print(type(s.index[-1]), s.index[-1], s[:s.index[-1] - pd.DateOffset(months=10)], (s.index[-1]+pd.DateOffset(months=1)) in s.index, (s.index[-1]+pd.DateOffset(months=3)) > (s.index[-1]+pd.DateOffset(months=2)))

s = data.loc[:, "Tasa oficial"]["MJJ2024"]
s = s.convert_dtypes('float64')

oa = OutlierAnalisys(
    time_serie=s,
    start_date=pd.to_datetime("2016-05-01", format="%Y-%m-%d"),
    end_date=pd.to_datetime("2018-09-01", format="%Y-%m-%d")
    )

oa.seasonality_diff(seasonal_model=x13_arima_analysis, mse_limit=oa.start)
oa.model_evolution()

KeyboardInterrupt: 

: 

In [21]:
oa.plot_evol()